## Cats and Dogs Classification Convolutional Neural Network Project

### Dataset

Dataset containing 10,000 images of cats and dogs. Dataset obtained from Kaggle at https://www.kaggle.com/datasets/tongpython/cat-and-dog. 

Dataset contained following (class / number of images):

- cat / 5,000
- dog / 5,000

##### Preprocessing

All images are resized to 244 x 244 in this notebook. The original dataset was separated into training and test folders of images, these folders were combined and the data was split in this notebook instead.

### Findings

From performing operations on this dataset, I have found that this convolutional neural network created to classify the cats and dogs was not accurate. As seen by the accuracy score of around 0.75, the model is correct in its classification more often than not, but is not consistent enough to be relied upon.

## Changelog

#### Version 1

- Set root directory for image locations
- Read in images.
- Resized images to all be the same size.
- Create ImageDataGenerator.
- Split data into training and validation data.
- Created  and compiled model.
- Trained model on the data.
- Analysed accuracy of the model using accuracy and loss.

In [32]:
# Import all necessary libraries

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import os
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.preprocessing import image

In [33]:
# Set root directory for class folders and target size for images

root_directory = 'catsanddogs/'
target_size = (224, 224)

In [34]:
# Resize images in dataset to target_size

for class_folder in os.listdir(root_directory):
    class_path = os.path.join(root_directory, class_folder)
    if os.path.isdir(class_path):  # Check if it is a directory
        # Loop through all images in the class folder
        for filename in os.listdir(class_path):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                img_path = os.path.join(class_path, filename)
                img = cv2.imread(img_path)
                img_resized = cv2.resize(img, target_size)
                cv2.imwrite(img_path, img_resized)  # Overwrite the original image or save to a new file

In [35]:
# Set number of training samples used in one iteration of training
# Set number of passes through dataset

batch_size = 128
epochs = 20

In [36]:
# Create ImageDataGenerator to allow real-time data augmentation. This helps the model generalise and reduces overfitting
# Rescale pixel values from 0-255 to 0-1 to improve convergence during training
# Reserve 20% of the dataset for validation

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [37]:
# Load training data

train_generator = train_datagen.flow_from_directory(
    root_directory,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 8023 images belonging to 2 classes.


In [38]:
# Check class indice values

print(train_generator.class_indices)

{'cats': 0, 'dogs': 1}


In [39]:
# Load validation data

validation_generator = train_datagen.flow_from_directory(
    root_directory,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 2005 images belonging to 2 classes.


In [40]:
# Three 2D convolution layers for RGB colours, filter size of 3x3
# Reduce height and width of feature maps to reduce number of parameters and computation in the network

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),   # flatten 3D output to 1D vector for following layers
    layers.Dense(128, activation='relu'),   # Learn high-level features by combining features learning in convolutional layers
    layers.Dense(len(train_generator.class_indices), activation='softmax')  # Same number of neurons as classes, softmax outputs probability for each class
])

In [41]:
# Adaptive Moment Estimation optimiser because it is memory-efficient and adapts the learning rate dynamically for each parameter
# Minimise loss function during training

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [42]:
# Train model

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)

Epoch 1/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 63s 997ms/step - accuracy: 0.5409 - loss: 1.1343 - val_accuracy: 0.6760 - val_loss: 0.6084
Epoch 2/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.6875 - loss: 0.5950 - val_accuracy: 0.6302 - val_loss: 0.6288
Epoch 3/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 37s 601ms/step - accuracy: 0.6952 - loss: 0.5826 - val_accuracy: 0.6896 - val_loss: 0.5866
Epoch 4/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.6875 - loss: 0.5947 - val_accuracy: 0.6885 - val_loss: 0.5801
Epoch 5/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 37s 601ms/step - accuracy: 0.7370 - loss: 0.5249 - val_accuracy: 0.6984 - val_loss: 0.5749
Epoch 6/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.7188 - loss: 0.5597 - val_accuracy: 0.7104 - val_loss: 0.5744
Epoch 7/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 37s 599ms/step - accuracy: 0.7804 - loss: 0.4680 - val_accuracy: 0.7406 - val_loss: 0.5227
Epoch 8/20
62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.8438 - loss: 0.4145 - val_accuracy: 0.

In [43]:
# Analyse loss and accuracy measurements

loss, accuracy = model.evaluate(validation_generator)
print(f'Validation loss: {loss}')
print(f'Validation accuracy: {accuracy}')

16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 152ms/step - accuracy: 0.7337 - loss: 0.9564
Validation loss: 0.9448468685150146
Validation accuracy: 0.7411471605300903


#### Results

An accuracy value of roughly 0.75 tells us that the model is correct more often than not, but not accurate enough to be relied upon consistently.

In [44]:
# Test model on image of sunflower

img_path = 'catsanddogs/cats/cat.1130.jpg'
img = image.load_img(img_path, target_size=target_size)
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)
print(f'Predicted class: {predicted_class}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Predicted class: [0]


In [45]:
# Test model on image of sunflower

img_path = 'catsanddogs/dogs/dog.290.jpg'
img = image.load_img(img_path, target_size=target_size)
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)
print(f'Predicted class: {predicted_class}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Predicted class: [1]
